In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import time
import pyproj
import glob

In [2]:
airport_location = pd.read_csv('../data/airport_positions.csv')

In [3]:
data = xr.open_mfdataset('/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/meps_lagged_6_h_subset_2_5km_20211005T00Z.nc', chunks={'time':1})
# Remove parenthesis for airport location in dataset -> split lon and lat values -> divide by 100 to obtain correct values
airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)
airport_location[['latitude', 'longitude']] = airport_location[['latitude', 'longitude']].astype('float') / 100

/tmp/ipykernel_1774595/4290719538.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  airport_location[['latitude', 'longitude']] = airport_location['position'].str.replace(r'[\(\)]', '').str.split(',', expand=True)


In [4]:
parameters = ['air_temperature_0m', 'air_temperature_2m']

In [5]:
def transform_latitude_longitude_to_xy(latitude_values, longitude_values):
    # Create a pyproj CRS object
    proj4_str = '+proj=lcc +lat_0=63.3 +lon_0=15 +lat_1=63.3 +lat_2=63.3 +no_defs +R=6.371e+06'
    lcc_crs = pyproj.CRS.from_proj4(proj4_str)

    # Create a transformer for converting between lat/lon and x/y
    transformer = pyproj.Transformer.from_crs(pyproj.CRS("EPSG:4326"), lcc_crs, always_xy=True)

    # Transform lat/lon to x/y
    x_values, y_values = transformer.transform(longitude_values, latitude_values)

    return x_values, y_values

# Extract from a specific file

In [6]:
latitude_values = airport_location.latitude.values
longitude_values = airport_location.longitude.values

x_values, y_values = transform_latitude_longitude_to_xy(latitude_values, longitude_values)

daily_airport_datasets = []

for airport_idx in range(len(latitude_values)):
    nearest_x = x_values[airport_idx]
    nearest_y = y_values[airport_idx]
    
    #print(f"AIRPORT {airport_idx + 1} COORDINATES:")
    #print(f"  Latitude: {latitude_values[airport_idx]}")
    #print(f"  Longitude: {longitude_values[airport_idx]}")
    #print(f"  Nearest x: {nearest_x}")
    #print(f"  Nearest y: {nearest_y}")
    
    # Extract latitude and longitude values from the data variable
    lat_from_data = data.latitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()
    lon_from_data = data.longitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()

    #print(f"  Latitude (Data): {lat_from_data}")
    #print(f"  Longitude (Data): {lon_from_data}")


    interpolated_data = data[parameters].interp(x=nearest_x, y=nearest_y)

    start_datetime = pd.to_datetime(data.time.values[0])  # Convert to pandas datetime
    end_datetime = start_datetime + pd.DateOffset(hours=5)  # Assuming each time step is 1 hour

    extracted_data = interpolated_data.sel(
        ensemble_member=0,
        time=slice(start_datetime, end_datetime)
    )
    # Add airport as a dimension with the corresponding index (+ 1)
    extracted_data = extracted_data.expand_dims({'airport': [airport_idx + 1]})
    daily_airport_datasets.append(extracted_data)
final_combined_data = xr.concat(daily_airport_datasets, dim='airport')

In [7]:
final_combined_data

<xarray.Dataset>
Dimensions:             (airport: 90, time: 6, height0: 1, height1: 1)
Coordinates:
  * airport             (airport) int64 1 2 3 4 5 6 7 8 ... 84 85 86 87 88 89 90
  * time                (time) datetime64[ns] 2021-10-05 ... 2021-10-05T05:00:00
    ensemble_member     int16 0
  * height0             (height0) float32 0.0
  * height1             (height1) float32 2.0
    longitude           (airport) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    latitude            (airport) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    x                   (airport) float64 -4.541e+05 4.509e+04 ... -5.383e+05
    y                   (airport) float64 -5.076e+04 6.677e+05 ... -4.537e+05
Data variables:
    air_temperature_0m  (airport, time, height0) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
    air_temperature_2m  (airport, time, height1) float32 dask.array<chunksize=(1, 1, 1), meta=np.ndarray>
Attributes: (12/14)
    Conventions:         CF-1.6
    institution:         Norwegian Meteorological Institute, MET Norway
    creator_url:         met.no
    source:              MEPS 2.5km
    min_time:            2021-10-07 12:00:00Z
    max_time:            2021-10-07
    ...                  ...
    geospatial_lon_min:  18.1
    geospatial_lon_max:  54.2
    references:          unknown
    license:             https://www.met.no/en/free-meteorological-data/Licen...
    comment:             none
    history:             2021-10-05T02:29:59 creation by fimex

# Expanded code: For all files in a day folder

In [23]:
# Define folder with paths and prefix for files to read
folder_path = '/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/'
file_prefix = 'meps_lagged_6_h_subset_2_5km_*.nc'
file_pattern = os.path.join(folder_path, file_prefix)
file_list = glob.glob(file_pattern)

In [25]:
all_timesteps_data = []
for file_path in file_list:
    data = xr.open_mfdataset(file_path, chunks={'time':1})
    
    print(file_path)
    
    latitude_values = airport_location.latitude.values
    longitude_values = airport_location.longitude.values

    x_values, y_values = transform_latitude_longitude_to_xy(latitude_values, longitude_values)

    airport_six_timesteps = []

    for airport_idx in range(len(latitude_values)):
        nearest_x = x_values[airport_idx]
        nearest_y = y_values[airport_idx]

        #print(f"AIRPORT {airport_idx + 1} COORDINATES:")
        #print(f"  Latitude: {latitude_values[airport_idx]}")
        #print(f"  Longitude: {longitude_values[airport_idx]}")
        #print(f"  Nearest x: {nearest_x}")
        #print(f"  Nearest y: {nearest_y}")

        # Extract latitude and longitude values from the data variable
        lat_from_data = data.latitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()
        lon_from_data = data.longitude.sel(x=nearest_x, y=nearest_y, method='nearest').values.item()

        #print(f"  Latitude (Data): {lat_from_data}")
        #print(f"  Longitude (Data): {lon_from_data}")


        interpolated_data = data[parameters].interp(x=nearest_x, y=nearest_y)

        start_datetime = pd.to_datetime(data.time.values[0])  # Convert to pandas datetime
        end_datetime = start_datetime + pd.DateOffset(hours=5)  # Assuming each time step is 1 hour

        extracted_data = interpolated_data.sel(
            ensemble_member=0,
            time=slice(start_datetime, end_datetime)
        )
        # Add airport as a dimension with the corresponding index (+ 1)
        extracted_data = extracted_data.expand_dims({'airport': [airport_idx + 1]})
        airport_six_timesteps.append(extracted_data)
    combined_data = xr.concat(airport_six_timesteps, dim='airport')
    all_timesteps_data.append(combined_data)
final_combined_data = xr.concat(all_timesteps_data, dim='time')


/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/meps_lagged_6_h_subset_2_5km_20211005T12Z.nc
/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/meps_lagged_6_h_subset_2_5km_20211005T00Z.nc
/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/meps_lagged_6_h_subset_2_5km_20211005T18Z.nc
/lustre/storeB/immutable/archive/projects/metproduction/meps/2021/10/05/meps_lagged_6_h_subset_2_5km_20211005T06Z.nc


In [31]:
final_combined_data

,Array,Chunk
Bytes,8.44 kiB,4 B
Shape,"(90, 24, 1)","(1, 1, 1)"
Count,3769 Graph Layers,2160 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,8.44 kiB,4 B
Shape,"(90, 24, 1)","(1, 1, 1)"
Count,3769 Graph Layers,2160 Chunks
Type,float32,numpy.ndarray
